In [1]:
import tensorflow as tf

2023-07-04 10:39:14.780328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 10:39:14.845801: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-04 10:39:14.863353: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-04 10:39:15.210342: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
print(f'tensorflow_version: {tf.__version__}')

tensorflow_version: 2.10.1


In [3]:
print("Tensorflow: Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Tensorflow: Num GPUs Available:  0


2023-07-04 10:39:21.659493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-04 10:39:21.659599: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-07-04 10:39:21.659618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-07-04 10:39:21.659633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-07-04 10:39:21.659648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

In [4]:

#activate conda env AutencoderTF env tf '2.10.0' , python 3.9.16
import tensorflow as tf
from skimage.metrics import mean_squared_error, normalized_root_mse,normalized_mutual_information
from sklearn.preprocessing import normalize, MinMaxScaler


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from alive_progress import  alive_bar
import torch
import sys
from pathlib import Path
import os

print(sys.path)
sys.path.append('/root/LatentOE-AD')
from loader.LoadData import CIFAR10_feat , FMNIST_feat
from Additional_Code_BULE.utils.helper_functions  import *
from Additional_Code_BULE.utils.AE_models_TF  import *


print(f'tensorflow_version: {tf.__version__}')
print(f'torch_version: {torch.__version__}')
print("Tensorflow: Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(f'GPU for pytorch: {torch.cuda.is_available()}')


DATA_PATH="/root/LatentOE-AD/DATA/fmnist_features/"
modelname='Autoencoder_features_simple_3'
MODEL_SHORT= 'AE3_derivatives'


['/root/LatentOE-AD/Additional_Code_BULE', '/root/LatentOE-AD/Additional_Code_BULE/c', '/root/LatentOE-AD/Additional_Code_BULE/\\\\Users\\\\buehl\\\\git\\\\projects\\\\LatentOE-AD', '/root/anaconda3/envs/AutencoderTF/lib/python39.zip', '/root/anaconda3/envs/AutencoderTF/lib/python3.9', '/root/anaconda3/envs/AutencoderTF/lib/python3.9/lib-dynload', '', '/root/anaconda3/envs/AutencoderTF/lib/python3.9/site-packages']
tensorflow_version: 2.10.1
torch_version: 2.0.1
Tensorflow: Num GPUs Available:  0
GPU for pytorch: True


In [ ]:




runs=[0,1,2,3,4]    #list of how many runs 
all_metrics=False # if True, all metrics are calculated, if False only mse 
testrun=False


labels=[0,1,2,3,4,5,6,7,8,9]
contam_list=np.round(np.arange(0,0.5,0.05),2)
epochs=10

if testrun:
    labels=[0]
    contam_list=[0.0,0.1]
    epochs=1

MODEL_RESULT_PATH = Path(f"/root/LatentOE-AD/RESULTS/fmnist/{MODEL_SHORT}")


def main():
    with alive_bar(len(runs)*len(labels)*len(contam_list)*10000,force_tty=True) as bar:
        #each run creates new samples
        for run in runs:
            # for every contamination ratio
            for contam in contam_list:

                nmi,mse,nrmse,csim,msle,normal_label,anomaly,contam_ratio=[],[],[],[],[],[],[],[]

                for i in labels:
                    # load data for each class
                    x_train, y_train, x_test, y_test = FMNIST_feat(i,root=DATA_PATH,contamination_rate=contam)
                    x_train=x_train.numpy()
                    x_test=x_test.numpy()

                    # # transformation of features train
                    scaler = MinMaxScaler()
                    scaler.fit(x_train)
                    x_train=np.sqrt(scaler.transform(x_train))

                    # # transformation of features train
                    scaler = MinMaxScaler()
                    scaler.fit(x_test)
                    x_test=np.sqrt(scaler.transform(x_test))
                    

                    #####  Autoencoder 3
                    # instantiate model Autoencoder
                    autoencoder = Autoencoder_features_simple_3()
                    autoencoder=autoencoder.model()
                
                    # fit to data
                    autoencoder.fit(x_train, x_train,epochs=epochs,shuffle=True,verbose=False)
                    decoded_imgs=autoencoder.predict(x_test,verbose=False)

                    ##### only for Autoencoder 1
                    # #testing on 1000 normal samples and 9000 anomalies per i
                    # encoded_imgs = autoencoder.encoder(x_test).numpy()
                    # decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

                    #reconstruction metrics for vectors
                    for j in range(0,len(decoded_imgs)):
                        
                        if all_metrics:
                            #different metrics 
                            mse.append(mean_squared_error(x_test[j],decoded_imgs[j]))
                            csim.append(tf.keras.losses.cosine_similarity(x_test[j],decoded_imgs[j]).numpy())
                            msle.append(tf.keras.losses.mean_squared_logarithmic_error(x_test[j],decoded_imgs[j]).numpy())
                            nrmse.append(normalized_root_mse(x_test[j],decoded_imgs[j]))
                            nmi.append(normalized_mutual_information(x_test[j],decoded_imgs[j])-1)
                        else:
                            mse.append(mean_squared_error(x_test[j],decoded_imgs[j]))

                        # labels ratio and anomaly ratio
                        normal_label.append(i)
                        anomaly.append(y_test[j])
                        contam_ratio.append(contam)
                        bar()
                if all_metrics:
                    df_per_contam =pd.DataFrame({'mse_': mse, 'csim_': csim,'msle_': msle,'nrmse_': nrmse,'normal_label':normal_label,'nmi_':nmi,'anomaly_':anomaly,'contam_ratio':contam_ratio})#,
                else:
                    df_per_contam =pd.DataFrame({'mse_': mse,'normal_label':normal_label,'anomaly_':anomaly,'contam_ratio':contam_ratio})#,

                SAVE_PATH=os.path.join(MODEL_RESULT_PATH,f"{modelname}_contam_{int(contam*100)}%_run_{run}.pkl")
                df_per_contam.to_pickle(SAVE_PATH)



            save_allresults_pickle(MODEL_RESULT_PATH,run=run,name=modelname)


if __name__ == "__main__":
    main()

In [ ]:
import tensorflow as tf

# Assuming you have an autoencoder model named 'autoencoder'
# and an input tensor named 'input_data'
autoencoder.trainable = False  # Set the model to inference mode
output = autoencoder(input_data)

# Calculate gradients of the output with respect to the input
grads = tf.gradients(output, input_data)[0]

# Create a TensorFlow session
with tf.compat.v1.Session() as sess:
    # Initialize variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # Retrieve the gradients of the input
    input_gradients = sess.run(grads)

In [27]:

DATA_PATH="/root/LatentOE-AD/DATA/fmnist_features/"
modelname='Autoencoder_features_simple_3'
MODEL_SHORT= 'AE3'
epochs=5


class Autoencoder_features_simple_3():
    # simple autoencoder 2 form https://towardsdatascience.com/anomaly-detection-using-autoencoders-5b032178a1ea
    
    def __init__(self,):
        #super(Autoencoder_features_simple_3, self).__init__()
        self.hidden_dim_1=1024
        self.hidden_dim_2=512
        self.input_dim=2048
        self.encoding_dim=64
        self.learning_rate=1e-7
        self.latentdim=64

    def model(self,):    
            input_layer = tf.keras.layers.Input(shape=(self.input_dim, ))
            #Encoder
            encoder = tf.keras.layers.Dense(self.input_dim, activation="tanh",activity_regularizer=tf.keras.regularizers.l2(self.learning_rate))(input_layer)
            encoder=tf.keras.layers.Dropout(0.2)(encoder)
            encoder = tf.keras.layers.Dense(self.hidden_dim_1, activation='relu')(encoder)
            encoder = tf.keras.layers.Dense(self.hidden_dim_2, activation=tf.nn.leaky_relu)(encoder)

            #latentdim
            latent = tf.keras.layers.Dense(self.latentdim, activation='relu')(encoder)

            # Decoder
            decoder = tf.keras.layers.Dense(self.hidden_dim_2, activation='relu')(latent)
            decoder=tf.keras.layers.Dropout(0.2)(decoder)
            decoder = tf.keras.layers.Dense(self.hidden_dim_1, activation='relu')(decoder)
            decoder = tf.keras.layers.Dense(self.input_dim, activation='tanh')(decoder)
            #Autoencoder
            autoencoder3 = tf.keras.Model(inputs=input_layer, outputs=decoder)

            autoencoder3.compile(optimizer='adam', loss=losses.MeanSquaredError())
            return(autoencoder3)




x_train, y_train, x_test, y_test = FMNIST_feat(0,root=DATA_PATH,contamination_rate=0.1)
x_train=x_train.numpy()
x_test=x_test.numpy()

# # transformation of features train
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train=np.sqrt(scaler.transform(x_train))

# # transformation of features train
scaler = MinMaxScaler()
scaler.fit(x_test)
x_test=np.sqrt(scaler.transform(x_test))


#####  Autoencoder 3
# instantiate model Autoencoder
autoencoder = Autoencoder_features_simple_3()
autoencoder=autoencoder.model()

# fit to data
autoencoder.fit(x_train, x_train,epochs=epochs,shuffle=True,verbose=False)
#decoded_imgs=autoencoder.predict(x_test,verbose=False)



In [24]:
# and an input tensor named 'input_data'
#autoencoder.trainable = False  # Set the model to inference mode

# Create a TensorFlow GradientTape context
with tf.GradientTape() as tape:
    # Forward pass
    output = autoencoder(x_test)

# Calculate gradients of the output with respect to the input
#input_gradients = tape.gradient(output, x_test)





In [34]:
epochs =10
# Load and preprocess the data
x_train, y_train, x_test, y_test = FMNIST_feat(0, root=DATA_PATH, contamination_rate=0.1)
x_train = x_train.numpy()
x_test = x_test.numpy()

# Scale the input features
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = np.sqrt(scaler.transform(x_train))

# Instantiate the autoencoder model
autoencoder = Autoencoder_features_simple_3()
autoencoder_model = autoencoder.model()

# Fit the model to the data
autoencoder_model.fit(x_train, x_train, epochs=epochs, shuffle=True, verbose=False)

# Calculate the derivatives
with tf.GradientTape() as tape:
    # Watch the input layer of the autoencoder model
    tape.watch(autoencoder_model.input)
    
    # Forward pass
    output = autoencoder_model(x_train)
    
# Calculate the derivatives of the output with respect to each input neuron
gradients = tape.gradient(output, autoencoder_model.input)

# Print the gradients for each input neuron
for i, gradient in enumerate(gradients[0]):
    print(f"Gradient of input neuron {i}: {gradient}")

TypeError: 'NoneType' object is not subscriptable

In [37]:
output

<tf.Tensor 'model_6/dense_48/Tanh:0' shape=(6666, 2048) dtype=float32>

In [26]:
tape.gradient(output, x_test)

AttributeError: 'numpy.ndarray' object has no attribute '_id'

In [32]:
x_test=tf.convert_to_tensor(x_test)


# Assuming you have an autoencoder model named 'autoencoder'
# and an input tensor named 'input_data'
autoencoder.trainable = False  # Set the model to inference mode

# Create a TensorFlow GradientTape
tape = tf.GradientTape()
tape.watch(x_test)

# Forward pass
output = autoencoder(x_test)

# Calculate gradients of the output with respect to the input
input_gradients = tape.gradient(output, x_test)

# Release the tape
del tape

AttributeError: 'NoneType' object has no attribute '_tape'